In [13]:
import cv2 ## for the webcam and computer vision
import numpy as np 
from playsound import playsound ## alert sound
from PIL import Image, ImageDraw
import face_recognition ## face recognition
from tensorflow import keras ## for loading in the model
import matplotlib.pyplot as plt

In [14]:
## make sure to specify the filepath to the model if needed
eye_model = keras.models.load_model('Version3Eye.h5')
lip_model = keras.models.load_model('Version2_lip.h5')

In [15]:
def lip_cropper(frame):
    facial_features_list = face_recognition.face_landmarks(frame)
    lips = []
    try:
        lips.append(facial_features_list[0]['top_lip'])
        lips.append(facial_features_list[0]['bottom_lip'])
    except:
        return

    x_max1 = max([coordinate[0] for coordinate in lips[0]])
    y_max1 = max([coordinate[1] for coordinate in lips[0]])
    x_min2 = min([coordinate[0] for coordinate in lips[1]])
    y_min2 = min([coordinate[1] for coordinate in lips[1]])
    x_range = x_max1 - x_min2
    y_range = y_max1 - y_min2
    #print(lips[1])
    #print(lips)
    #print(x_max1,x_min1,y_max1,y_min1)
    #print(x_range,y_range)

    if x_range > y_range:
        right = int(round(.3*x_range) + x_max1)
        left = int(x_min2 - round(.3*x_range))
        bottom = int(round(((right-left) - y_range))/2 + y_max1)
        top = int(y_min2 - round(((right-left) - y_range))/2)
    else:
        bottom = int(round(.3*y_range) + y_max1)
        top = int(y_min2 - round(.3*y_range))
        right = int(round(((bottom-top) - x_range))/2 + x_max1)
        left = int(x_min2 - round(((bottom-top) - x_range))/2)

    #print(bottom,top,right,left)
    cropped = frame[top:(bottom + 1), left:(right + 1)]
    # resize the image

    #cropped = cv2.resize(cropped, (32,32))
    #image_for_prediction = cropped.reshape(-1, 32, 32, 3)
    image_for_prediction = cv2.resize(cropped, (80,80))


    return image_for_prediction



In [16]:
def eye_cropper(frame):

    # create a variable for the facial feature coordinates

    facial_features_list = face_recognition.face_landmarks(frame)


    # create a placeholder list for the eye coordinates
    # and append coordinates for eyes to list unless eyes
    # weren't found by facial recognition

    try:
        eye = facial_features_list[0]['left_eye']
    except:
        try:
            eye = facial_features_list[0]['right_eye']
        except:
            return


    # establish the max x and y coordinates of the eye

    x_max = max([coordinate[0] for coordinate in eye])
    x_min = min([coordinate[0] for coordinate in eye])
    y_max = max([coordinate[1] for coordinate in eye])
    y_min = min([coordinate[1] for coordinate in eye])


    # establish the range of x and y coordinates

    x_range = x_max - x_min
    y_range = y_max - y_min


      # in order to make sure the full eye is captured,
      # calculate the coordinates of a square that has a
      # 50% cushion added to the axis with a larger range and
      # then match the smaller range to the cushioned larger range

    if x_range > y_range:
        right = round(.5*x_range) + x_max
        left = x_min - round(.5*x_range)
        bottom = round((((right-left) - y_range))/2) + y_max
        top = y_min - round((((right-left) - y_range))/2)
    else:
        bottom = round(.5*y_range) + y_max
        top = y_min - round(.5*y_range)
        right = round((((bottom-top) - x_range))/2) + x_max
        left = x_min - round((((bottom-top) - x_range))/2)


    # crop the image according to the coordinates determined above

    cropped = frame[top:(bottom + 1), left:(right + 1)]

    # resize the image

    #cropped = cv2.resize(cropped, (32,32))
    #image_for_prediction = cropped.reshape(-1, 32, 32, 3)
    image_for_prediction = cv2.resize(cropped, (80,80))


    return image_for_prediction

In [17]:
BUFFER_AMOUNT = 20

def Drowsy_Studying():
    # initiate webcam
    eye = 0
    lip = 0
    cap = cv2.VideoCapture(0)
    w = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    h = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    #print(cap.get(cv2.CAP_PROP_FPS))


    if not cap.isOpened():
        raise IOError('Cannot open webcam')

    # set a counter
    score = 100
    counter = 0
    rounds = 0
    Mark = 0

    # create a while loop that runs while webcam is in use

    while Mark < 5:

        # capture frames being outputted by webcam
        
        ret, frame = cap.read()

        # use only every other frame to manage speed and memory usage

        frame_count = 0
        if frame_count == 0:
            frame_count += 1
            pass
        else:
            count = 0
            continue

        # function called on the frame
        try:
            image_for_prediction = eye_cropper(frame)
            image_for_prediction_lip = lip_cropper(frame)
            image_for_prediction = np.array(image_for_prediction)
            image_for_prediction = np.expand_dims(image_for_prediction, axis=0)
            image_for_prediction_lip = np.array(image_for_prediction_lip)
            image_for_prediction_lip = np.expand_dims(image_for_prediction_lip, axis=0)
            prediction = eye_model.predict(image_for_prediction)
            prediction_lip = lip_model.predict(image_for_prediction_lip)
            prediction = np.argmax(prediction[0], axis=0)
            prediction_lip = np.argmax(prediction_lip[0], axis=0)
        except:
            continue


        if prediction == 1:
            eye = eye + 1
        else: 
            eye = 0
        if prediction_lip == 0:
            lip = lip + 1
        else: 
            lip = 0
        # Based on prediction, display either "Open Eyes" or "Closed Eyes"
        try:
            if lip <2 and eye < 5:
                counter = 0
                status = 'No_Yawn + Eye_Open'
                score -=  0.1

            elif lip > 2 and eye > 5:
                counter = counter + 1
                status = 'Yawn + Eye_closed'
                score -= 5

            elif lip < 2 and eye > 5:
                counter = counter + 1
                status = 'No_Yawn + Eye_closed'
                score -= 4

            else: 
                counter = counter + 1
                status = 'Yawn + Eye_Open'
                score -= 3

        except:
            continue
        rounds +=1 
        #print(status)
        if rounds > 20:
            if score < 0:
                score = 0
            print("Score is {0}".format(score), end=' \n')
            rounds = 0
            score = 100
            Mark +=1
            
        #cv2.imshow('Drowsiness Detection', frame)
        #print(rounds)
        k = cv2.waitKey(1)
        if k == 27:
            break
    cap.release()
    cv2.destroyAllWindows()

In [18]:
Drowsy_Studying()

Score is 87.2000000000001 
Score is 52.10000000000005 
Score is 16 
Score is 58.899999999999984 
Score is 45.30000000000004 
